In [1]:
import pandas as pd
import numpy as np
import random
import datetime

https://leetcode.com/problems/percentage-of-users-attended-a-contest/?lang=pythondata

In [2]:
data = [[6, 'Alice'], [2, 'Bob'], [7, 'Alex']]
users = pd.DataFrame(data, columns=['user_id', 'user_name']).astype({'user_id':'Int64', 'user_name':'object'})
data = [[215, 6], [209, 2], [208, 2], [210, 6], [208, 6], [209, 7], [209, 6], [215, 7], [208, 7], [210, 2], [207, 2], [210, 7]]
register = pd.DataFrame(data, columns=['contest_id', 'user_id']).astype({'contest_id':'Int64', 'user_id':'Int64'})
users

,user_id,user_name
0,6,Alice
1,2,Bob
2,7,Alex


In [3]:
register

,contest_id,user_id
0,215,6
1,209,2
2,208,2
3,210,6
4,208,6
5,209,7
6,209,6
7,215,7
8,208,7
9,210,2


pandas.DataFrame.rename(mapper=None, *, index=None, columns=None, axis=None, copy=None, inplace=False, level=None, errors='ignore') : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html

* DataFrame에서 column이나 index의 이름을 바꾸고 싶을 때 사용
* ***mapper***는 DataFrame의 index나 column label을 바꿀 때, 쓸 함수를 넣어주는 것임. 이 mapper로 쓰게 될 함수의 매개변수로 DataFrame의 모든 index나 column이 하나씩 들어가게 되고, mapper함수에 의해 각각이 변경된다.
* ***columns***는 DataFrame의 column label을 어떤 것으로 바꿔줄 것인지 dict형태로 넣어준다.
* ***axis***는 바꿀 축을 정한다. 1이면 column, 0이면 index를 바꾼다는 것을 의미한다. 숫자대신 column, index 이렇게 써도 된다.

In [25]:
# column label이 기존 A, B에서 a, b로 바뀐다.
ddf = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
ddf.rename(columns={"A": "a", "B": "c"})
ddf

,A,B
0,1,4
1,2,5
2,3,6


In [32]:
# mapper 매개변수에 lambda를 사용해서 column의 label들을 전부 바꾸었다.
# axis를 columns으로 뒀기 때문에 가능한 일이다.
ddf = pd.DataFrame({"col1": ['a', 'b', 'c'], "col2": [1, 2, 3]})
ddf.rename(mapper=lambda x: "abc_"+x, axis='columns')

,abc_col1,abc_col2
0,a,1
1,b,2
2,c,3


In [16]:
# 직관적인 풀이
total_users = len(users['user_id'].unique())
num_lst = []
id_lst = register['contest_id'].unique()
for contest_id in id_lst:
  num_lst.append(round((len(register[register['contest_id']==contest_id]) / total_users)*100, 2))
dic = {'contest_id':id_lst, 'percentage':num_lst}
df = pd.DataFrame(dic)
df.sort_values(by=['percentage', 'contest_id'], ascending=[False, True])

,contest_id,percentage
2,208,100.00
1,209,100.00
3,210,100.00
0,215,66.67
4,207,33.33


In [22]:
group = register.groupby('contest_id', as_index=False).count().rename(columns={'user_id':'count'})
group

,contest_id,count
0,207,1
1,208,3
2,209,3
3,210,3
4,215,2


In [28]:
# groupby를 활용한 풀이
for contest_id in register['contest_id']:
  group = group.assign(percentage=round((group['count']/len(users['user_id'].unique())*100), 2))
group.sort_values(by=['percentage','contest_id'], ascending=[False, True]).drop(columns=['count'])

,contest_id,percentage
1,208,100.00
2,209,100.00
3,210,100.00
4,215,66.67
0,207,33.33


In [34]:
grouped = register.groupby("contest_id")["user_id"].count().reset_index(name="count")
grouped

,contest_id,count
0,207,1
1,208,3
2,209,3
3,210,3
4,215,2


In [33]:
# 비슷한데 다른 풀이.
grouped = register.groupby("contest_id")["user_id"].count().reset_index(name="count")
# percentage라는 새 column을 만들고 거기에 값으로 count Series 각각의 원소들에 apply를 적용한 결과를 넣는다.
grouped["percentage"] = grouped["count"].apply(lambda count: round(count/len(users) * 100,2))
grouped.drop(columns=["count"], inplace= True)
grouped.sort_values(by=["percentage", "contest_id"], ascending=[False, True], inplace= True)
grouped

,contest_id,percentage
1,208,100.00
2,209,100.00
3,210,100.00
4,215,66.67
0,207,33.33
